# 枚举

## C风格的枚举

In [4]:
#[derive(Debug)]
enum MyOrdering {
    Less,
    Equal,
    Greater
}


fn compare(n: i32, m: i32) -> MyOrdering {
    if n < m {
        MyOrdering::Less
    } else if n > m {
        MyOrdering::Greater
    } else {
        MyOrdering::Equal
    }
}

{
    println!("{:?}", compare(10, 20));
}


Less


()

In [6]:
enum HttpStatus { // C Style
    Ok = 200,
    NotModified = 304,
    NotFound = 404
}

fn http_status_from_u32(n: u32) -> Option<HttpStatus> {
    match n {
        200 => Some(HttpStatus::Ok),
        304 => Some(HttpStatus::NotModified),
        404 => Some(HttpStatus::NotFound),
        _ => None
    }
}

In [12]:
// 枚举方法

#[derive(Copy, Clone, Debug, PartialEq)]
enum TimeUnit {
    Seconds, Minutes, Hours, Days, Months, Years
}

impl TimeUnit {
    fn plural(self) -> &'static str {
        match self {
            TimeUnit::Seconds => "Seconds",
            TimeUnit::Minutes => "Minutes",
            TimeUnit::Hours => "Hours",
            TimeUnit::Days => "Days",
            TimeUnit::Months => "Months",
            TimeUnit::Years => "Years"
        }
    }
    
    fn singular(self) -> &'static str {
        self.plural().trim_end_matches('s')
    }
}


{
    println!("{:?}", TimeUnit::Hours.plural());
}

"Hours"


()

## 带数据的枚举

In [10]:
#[derive(Copy, Clone, Debug, PartialEq)]
enum RoughTime {
    InThePast(TimeUnit, u32),
    JustNow,
    InTheFuture(TimeUnit, u32)
}

{
    let four_score_and_seven_years_ago = RoughTime::InThePast(TimeUnit::Years, 4*20+7);
    let three_hours_from_now = RoughTime::InTheFuture(TimeUnit::Hours, 3);
    
    println!("{:?}", four_score_and_seven_years_ago);
    println!("{:?}", three_hours_from_now);
}

InThePast(Years, 87)
InThePast(Years, 87)
InTheFuture(Hours, 3)


()

### BinaryTree

In [23]:
#[derive(Debug)]
enum BinaryTree<T> {
    Empty,
    NonEmpty(Box<TreeNode<T>>)
}

#[derive(Debug)]
struct TreeNode<T> {
    element: T,
    left: BinaryTree<T>,
    right: BinaryTree<T>
}


impl<T: Ord> BinaryTree<T> {
    fn add(&mut self, value: T) {
        match *self {
            BinaryTree::Empty => 
                *self = BinaryTree::NonEmpty(Box::new(TreeNode{
                    element: value,
                    left: BinaryTree::Empty,
                    right: BinaryTree::Empty        
            })),
            BinaryTree::NonEmpty(ref mut node) =>  // 匹配引用
                if value <= node.element {
                    node.left.add(value);
                } else {
                    node.right.add(value);
                }
        }
    }
}

{
    use BinaryTree::*;
    let jupiter_tree = NonEmpty(Box::new(TreeNode {
        element: "Jupiter",
        left: Empty,
        right: Empty
    }));
    let mercury_tree = NonEmpty(Box::new(TreeNode {
        element: "Mercury",
        left: Empty,
        right: Empty
    }));    
    
    let mars_tree = NonEmpty(Box::new(TreeNode {
        element: "Mars",
        left: jupiter_tree,
        right: mercury_tree
    }));
    
    let mut tree = BinaryTree::Empty;
    tree.add("Jupiter");
    tree.add("Mercury");
    tree.add("Mars");
    println!("{:#?}", tree); // Pretty output: {:#?}
}

NonEmpty(
    TreeNode {
        element: "Jupiter",
        left: Empty,
        right: NonEmpty(
            TreeNode {
                element: "Mercury",
                left: NonEmpty(
                    TreeNode {
                        element: "Mars",
                        left: Empty,
                        right: Empty,
                    },
                ),
                right: Empty,
            },
        ),
    },
)


()

### JSON

In [14]:
use std::collections::HashMap;

enum Json {
    Null,
    Boolean(bool),
    Number(f64),
    String(String),
    Array(Vec<Json>),
    Object(Box<HashMap<String, Json>>)
}

# 模式

In [25]:
fn rough_time_to_english(rt: RoughTime) -> String {
    match rt {
        RoughTime::InThePast(units, count) =>
            format!("{} {} ago", count, units.plural()),
        RoughTime::JustNow =>
            format!("just now"),
        RoughTime::InTheFuture(units, count) =>
            format!("{} {} from now", count, units.plural())
    }
}

{
    println!("{:?}", rough_time_to_english(RoughTime::InTheFuture(TimeUnit::Days, 2)));
}

"2 Days from now"


()

## 字面量模式

In [26]:
{
    let n = 1;
    match n {
        0 => {},
        1 => println!("1"),
        n => println!("{}", n)
    }
}

1


()

In [41]:
#[derive(Debug, PartialEq)]
struct Hex { x: i32 };
struct Point { x: f32, y: f32 };

fn point_to_hex(click: Point) -> Option<Hex> {
    Some(Hex { x: 16 })
}

fn check_move(current_hex: Hex, click: Point) -> Result<Hex, &'static str> {
    match point_to_hex(click) {
        None => Err("wrong point"),
        Some(hex) => 
            if hex == current_hex {
                Err("same point")
            } else {
                Ok(hex)
            }
    }
}


{
    let result = check_move(Hex{x: 17}, Point { x: 1.0, y: 1.0 });
    match result {
        Ok(hex) => println!("{:?}", hex),
        Err(msg) => println!("Error: {}", msg)
    }
}

Hex { x: 16 }


## 元组和结构体模式

In [44]:
fn describe_point(x: i32, y: i32) -> &'static str {
    use std::cmp::Ordering::*;
    
    match (x.cmp(&0), y.cmp(&0)) {
        (Equal, Equal) => "at the origin",
        (_, Equal) => "on the x axis",
        (Equal, _) => "on the y axis",
        (Greater, Greater) => "in the first quadrant",
        (Less, Greater) => "int the second quadrant",
        _ => "somewhere else"
    }
}

{
    println!("{}", describe_point(10, 1));
}


in the first quadrant


()

In [52]:
#[derive(Debug)]
struct Account {
    name: String,
    language: String,
    id: u64,
    status: i32,
    address: String
}

impl Account {
    fn new_random() -> Option<Account> {
        Some(Account {
            name: "NAME".to_string(),
            language: "CN".to_string(),
            id: 0,
            status: 0,
            address: "ADDRESS".to_string()
        })
    }
}

{
    match Account::new_random() {
        Some(Account {name, language, ..}) => println!("name={}, language={}", name, language),
        _ => {}
    }
}

name=NAME, language=CN


()

## 引用模式

In [58]:
{
    let account = Account {            
        name: "NAME".to_string(),            
        language: "CN".to_string(),            
        id: 0,            
        status: 0,    
        address: "ADDRESS".to_string()    
    };
    
    match account {
        Account {ref name, ref language, ..} => {  // 引用模式
            println!("name={}, language={}", name, language);
            println!("account={:?}", account)
        }
    }
    
//     match account {
//         Account {name, language, ..} => {          // 移动
//             println!("name={}, language={}", name, language);
//             println!("account={:?}", account)
//         }
//     }
}

name=NAME, language=CN
account=Account { name: "NAME", language: "CN", id: 0, status: 0, address: "ADDRESS" }


()

## 多个模式

In [61]:
{
    let next_char = 'c';
    
    match next_char {
        '0' ..= '9' => println!("digit"),
        'a' ..= 'z' | 'A' ..= 'Z' => println!("character"),
        ' ' | '\t' | '\n' => println!("whitespace"),
        _ => println!("unknown")
    }
}

character


()

## 模式护卫

In [62]:
{
    let n = 10;
    
    match n {
        var if var > 20 => println!("{} is greater than 20", var),
        var => println!("{}", var)
    }
}

10


()

## `@`模式

In [74]:
#![feature(or_patterns)]

{
    let next_char = 'c';
    
    match next_char {
        digit @ '0' ..= '9' => println!("digit={}", digit),
        // character @ ('a' ..= 'z' | 'A' ..= 'Z') => println!("character={}", character),
        'a' ..= 'z' | 'A' ..= 'Z' => println!("character={}", next_char),
        ' ' | '\t' | '\n' => println!("whitespace"),
        _ => println!("unknown")
    }
}

character=c
